In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Conv1D, BatchNormalization, Dropout, Flatten, Dense, Input,
                                    LSTM, Bidirectional, MultiHeadAttention, LayerNormalization, MaxPooling1D)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

In [4]:
# Load dataset
data = pd.read_csv("data.csv")
print(data.isnull().sum())  # Check for missing values
data = data.dropna()
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

Unnamed: 0    0
X1            0
X2            0
X3            0
X4            0
             ..
X175          0
X176          0
X177          0
X178          0
y             0
Length: 180, dtype: int64


In [6]:
# Preprocess labels
y = np.where(y == 1, 1, 0)
y = to_categorical(y)

In [7]:
# Normalize and reshape
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [8]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y.argmax(axis=1)), y=y.argmax(axis=1))
class_weight_dict = dict(enumerate(class_weights))

In [10]:
# Define Transformer Encoder Block
def transformer_block(x, num_heads, key_dim, ff_dim=256):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
    attn_output = LayerNormalization(epsilon=1e-6)(x + attn_output)
    ffn_output = Dense(ff_dim, activation='relu')(attn_output)
    ffn_output = Dense(x.shape[-1])(ffn_output)
    ffn_output = LayerNormalization(epsilon=1e-6)(attn_output + ffn_output)
    return ffn_output

In [11]:
# Build Model
input_layer = Input(shape=(X_train.shape[1], 1))
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.3)(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Dropout(0.3)(x)
x = transformer_block(x, num_heads=4, key_dim=64, ff_dim=256)
x = Flatten()(x)
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
output_layer = Dense(2, activation='softmax')(x)

In [12]:
# Compile Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [14]:
# Train Model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32,
          class_weight=class_weight_dict, callbacks=[early_stopping, lr_scheduler], verbose=1)

Epoch 1/50
288/288 ━━━━━━━━━━━━━━━━━━━━ 45s 135ms/step - accuracy: 0.9326 - loss: 2.4006 - val_accuracy: 0.9148 - val_loss: 0.6749 - learning_rate: 0.0010
Epoch 2/50
288/288 ━━━━━━━━━━━━━━━━━━━━ 38s 133ms/step - accuracy: 0.9505 - loss: 0.5201 - val_accuracy: 0.9804 - val_loss: 0.3080 - learning_rate: 0.0010
Epoch 3/50
288/288 ━━━━━━━━━━━━━━━━━━━━ 42s 137ms/step - accuracy: 0.9635 - loss: 0.2985 - val_accuracy: 0.9887 - val_loss: 0.1625 - learning_rate: 0.0010
Epoch 4/50
288/288 ━━━━━━━━━━━━━━━━━━━━ 40s 132ms/step - accuracy: 0.9779 - loss: 0.1771 - val_accuracy: 0.9826 - val_loss: 0.1415 - learning_rate: 0.0010
Epoch 5/50
288/288 ━━━━━━━━━━━━━━━━━━━━ 41s 132ms/step - accuracy: 0.9803 - loss: 0.1331 - val_accuracy: 0.9843 - val_loss: 0.1138 - learning_rate: 0.0010
Epoch 6/50
288/288 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.9815 - loss: 0.1194 - val_accuracy: 0.9343 - val_loss: 0.2098 - learning_rate: 0.0010
Epoch 7/50
288/288 ━━━━━━━━━━━━━━━━━━━━ 42s 132ms/step - accuracy: 0.9

In [15]:
# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
y_pred = model.predict(X_test).argmax(axis=1)
y_true = y_test.argmax(axis=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision_score(y_true, y_pred):.2f}")
print(f"Recall: {recall_score(y_true, y_pred):.2f}")
print(f"F1-Score: {f1_score(y_true, y_pred):.2f}")

72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step
Test Accuracy: 99.74%
Precision: 0.99
Recall: 1.00
F1-Score: 0.99
